# Let's bring in all the scraped data

In [129]:
import warnings
warnings.filterwarnings('ignore')

In [130]:
import csv
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pprint
import nltk
from nltk.stem.porter import PorterStemmer

In [131]:
# read in all the job postings

corpusDF = pd.DataFrame()

for f in os.listdir("collected_data/"):
    if (f.endswith(".csv")):
        
        print("Importing... ", f)
        
        filepath = os.path.join("collected_data", f)
        
        df = pd.read_csv(filepath, encoding = "ISO-8859-1")
        
        df["AllDescription"] = df["PositionTitle"] + " | " +  \
                                df["URI"] + " | " + df["DepartmentName"]  + " | " + \
                                df["OrganizationName"] + " | " + f  + " | " + \
                                df["JobSummary"] + " | " + \
                                df["QualificationSummary"]
        
        corpusDF = corpusDF.append(df, ignore_index = True)        
        
        print("Success")
        print("================================================================")

Importing...  aerospace_N_113_20171224133539.csv
Success
Importing...  analysis_N_500_20171223141842.csv
Success
Importing...  analyst_N_239_20171223115544.csv
Success
Importing...  attorney_N_180_20171223141257.csv
Success
Importing...  aviation_N_355_20171224133743.csv
Success
Importing...  businessintelligence_N_20_20171223115447.csv
Success
Importing...  combat_N_400_20171224133814.csv
Success
Importing...  computational_N_17_20171223115812.csv
Success
Importing...  covert_N_4_20171223141634.csv
Success
Importing...  cyber_N_76_20171223141650.csv
Success
Importing...  dataanalyst_N_96_20171223115356.csv
Success
Importing...  dataanalytics_N_19_20171223115427.csv
Success
Importing...  database_N_500_20171223120145.csv
Success
Importing...  dataengineer_N_101_20171223115828.csv
Success
Importing...  datamining_N_9_20171223120312.csv
Success
Importing...  datascience_N_346_20171223115408.csv
Success
Importing...  datascientist_N_28_20171223115414.csv
Success
Importing...  datavisualiz

Success
Importing...  soldier_N_162_20171224133432.csv
Success
Importing...  spatial_N_14_20171223115245.csv
Success
Importing...  SPSS_N_3_20171223120435.csv
Success
Importing...  sql_N_21_20171223120239.csv
Success
Importing...  statistical_N_277_20171223120834.csv
Success
Importing...  statistician_N_8_20171223115854.csv
Success
Importing...  statistics_N_329_20171223115916.csv
Success
Importing...  strategic_N_339_20171224133904.csv
Success
Importing...  surgeon_N_58_20171224133621.csv
Success
Importing...  systems_N_500_20171223120219.csv
Success
Importing...  tableau_N_2_20171223120948.csv
Success
Importing...  tactical_N_142_20171224133851.csv
Success
Importing...  technical_N_500_20171223115709.csv
Success
Importing...  technician_N_500_20171223141315.csv
Success
Importing...  technology_N_500_20171223120810.csv
Success
Importing...  transportation_N_500_20171224133801.csv
Success
Importing...  weaponry_N_17_20171224133630.csv
Success


In [132]:
corpusDF.head(3)

,ApplicationCloseDate,DepartmentName,JobGrade,JobSummary,MaxPay,MinPay,OfferingType,OrganizationName,PayType,PositionEndDate,PositionID,PositionLocation,PositionStartDate,PositionTitle,QualificationSummary,URI,AllDescription
0,2018-01-08,National Aeronautics and Space Administration,GS,The Applied Engineering and Technology Directo...,86460.0,66510.0,Permanent,Goddard Space Flight Center,Per Year,2018-01-08,GS18D0058,"Greenbelt, Maryland",2017-12-20,"Aerospace Engineer, AST, Aerospace Vehicle Des...",In addition to the Basic Education Requirement...,https://www.usajobs.gov:443/GetJob/ViewDetails...,"Aerospace Engineer, AST, Aerospace Vehicle Des..."
1,2017-12-28,Department of the Navy,GS,The selectee for this position will serve as a...,72901.0,45970.0,Permanent,"U.S. Atlantic Fleet, Commander in Chief",Per Year,2017-12-28,DE-10078062-18-HB,"Point Mugu, California",2017-12-18,AEROSPACE ENGINEER,"In order to qualify for this position, your re...",https://www.usajobs.gov:443/GetJob/ViewDetails...,AEROSPACE ENGINEER | https://www.usajobs.gov:4...
2,2017-12-30,Department of the Air Force,GS,The mission of the United States Air Force is ...,86460.0,45970.0,Permanent,"Air Force Elements, U.S. Strategic Command",Per Year,2017-12-30,9L-10088723-064321,"Dahlgren, Virginia",2017-12-01,Aerospace Engineer,GS-0861-07 Aerospace Engineer:\r\nA Bachelor&#...,https://www.usajobs.gov:443/GetJob/ViewDetails...,Aerospace Engineer | https://www.usajobs.gov:4...


## Removing duplicate job postings based on URI

In [133]:
len(corpusDF)

18426

In [134]:
corpusDF = corpusDF.groupby("URI").first().reset_index()
len(corpusDF)

5904

In [135]:
corpusDF.head(3)

,URI,ApplicationCloseDate,DepartmentName,JobGrade,JobSummary,MaxPay,MinPay,OfferingType,OrganizationName,PayType,PositionEndDate,PositionID,PositionLocation,PositionStartDate,PositionTitle,QualificationSummary,AllDescription
0,https://www.usajobs.gov:443/GetJob/ViewDetails...,2018-02-15,Other Agencies and Independent Organizations,GS,Thisfile serves as a repository of resumes for...,82642.00,20527.00,Multiple Appointment Types,Office of Personnel Management,Per Year,2018-02-15,US OPM Veteran File,"San Francisco, California|Denver, Colorado|Was...",2017-02-16,U.S. OPM Veteran File,As positions become available and hiring offic...,U.S. OPM Veteran File | https://www.usajobs.go...
1,https://www.usajobs.gov:443/GetJob/ViewDetails...,2018-01-03,Department of the Air Force,WG,Air Reserve Technicians are part of the regula...,26.18,22.45,Multiple Appointment Types,"Headquarters, Air Force Reserve Command",Per Hour,2018-01-03,SEU-SR-Elec Int Sys Mech,"Elmendorf AFB, Alaska|Maxwell AFB, Alabama|Lit...",2017-01-04,Electronic Integrated System Mechanic (Militar...,A specific length of experience or training is...,Electronic Integrated System Mechanic (Militar...
2,https://www.usajobs.gov:443/GetJob/ViewDetails...,2018-01-03,Department of the Air Force,WG,Air Reserve Technicians are part of the regula...,26.81,22.99,Multiple Appointment Types,"Headquarters, Air Force Reserve Command",Per Hour,2018-01-03,SEU-SR-Aircraft Engine Mech,"Elmendorf AFB, Alaska|Maxwell AFB, Alabama|Lit...",2017-01-04,Aircraft Engine Mechanic (Military Requirement),A specific length of experience or training is...,Aircraft Engine Mechanic (Military Requirement...


## Top 10 departments with the highest average paying jobs

In [136]:
set(corpusDF["PayType"])

{'Per Hour', 'Fee Basis', 'Without Compensation', 'Student Stipend Paid', 'Bi-weekly', 'Per Year', 'Per Day'}

In [137]:
print("Total Jobs: ", len(corpusDF))

Total Jobs:  5904


**For ease of analysis, I'm only going to look at "Per Year" jobs with the assumption that these salaried positions typically pay higher than the hourly positions. **

In [138]:
salaried = corpusDF[corpusDF["PayType"] == "Per Year"]
print("Total Salaried Jobs: ", len(salaried))

Total Salaried Jobs:  4291


**There are also some positions with multiple location placements. I'm going to filter those out. They have a single pipe delimitor in the "PositionLocation" column. **

In [139]:
salaried["hasMultipleLocations"] = salaried["PositionLocation"].apply(lambda row: True if "|" in row else False)
salaried_location = salaried[salaried["hasMultipleLocations"] == False]

In [140]:
print("Total Salaried Single Location Jobs: ", len(salaried_location))

Total Salaried Single Location Jobs:  3402


** I also want to only examine departments with at least 10 roles available. Investigating a department with only 2 roles available that pays a lot doesn't tell me much **

In [141]:
count = salaried_location.groupby("DepartmentName")["DepartmentName"].count().to_frame(name = "Counts").reset_index()
interestedDepts = count[count["Counts"] >= 10].sort_values("Counts", ascending = False).reset_index(drop = True)
interestedDepts

,DepartmentName,Counts
0,Department of Veterans Affairs,1188
1,Department of the Army,663
2,Department of the Air Force,296
3,Department of Health And Human Services,213
4,Other Agencies and Independent Organizations,193
5,Department of Defense,183
6,Department of the Navy,158
7,Department of Homeland Security,104
8,Department of Commerce,71
9,Department of the Interior,60


In [142]:
interestedDepts = np.array(interestedDepts["DepartmentName"])
interestedDepts

array(['Department of Veterans Affairs', 'Department of the Army',
       'Department of the Air Force',
       'Department of Health And Human Services',
       'Other Agencies and Independent Organizations',
       'Department of Defense', 'Department of the Navy',
       'Department of Homeland Security', 'Department of Commerce',
       'Department of the Interior', 'Department of Justice',
       'National Aeronautics and Space Administration',
       'Legislative Branch', 'Department of Transportation',
       'Department of Agriculture', 'Department of the Treasury',
       'Department of Energy', 'Judicial Branch'], dtype=object)

In [143]:
len(salaried_location)

3402

In [144]:
salaried_location = salaried_location[salaried_location["DepartmentName"].isin(interestedDepts)]

In [145]:
len(salaried_location)

3384

**Calculate the MedPay**

In [146]:
salaried_location["MedPay"] = salaried_location[["MinPay", "MaxPay"]].mean(axis = 1)
salaried_location.head(3)

,URI,ApplicationCloseDate,DepartmentName,JobGrade,JobSummary,MaxPay,MinPay,OfferingType,OrganizationName,PayType,PositionEndDate,PositionID,PositionLocation,PositionStartDate,PositionTitle,QualificationSummary,AllDescription,hasMultipleLocations,MedPay
30,https://www.usajobs.gov:443/GetJob/ViewDetails...,2018-03-01,Department of Veterans Affairs,GS,NOTE:&#160; If you apply for this position ele...,92316.0,71012.0,Multiple Appointment Types NTE NTE 2 years,"Veterans Affairs, Veterans Health Administration",Per Year,2018-03-01,OB-16-YC-1642529-RES-DH,"Birmingham, Alabama",2017-03-02,Health Science Specialist,"To qualify for this position, applicants must ...",Health Science Specialist | https://www.usajob...,False,81664.0
31,https://www.usajobs.gov:443/GetJob/ViewDetails...,2018-06-12,Other Agencies and Independent Organizations,GS,"Join SSA, one of the top 10 Best Places to Wor...",57090.0,32318.0,Agency Employees Only,Social Security Administration,Per Year,2018-06-12,Peace Corp CR 5/7 ACWA Test,"Chicago, Illinois",2017-06-13,Social Insurance Specialist (Claims Specialist),All qualification requirements must be met by ...,Social Insurance Specialist (Claims Specialist...,False,44704.0
32,https://www.usajobs.gov:443/GetJob/ViewDetails...,2017-12-29,Department of Veterans Affairs,VM,OUR MISSION: To fulfill President Lincoln's\r\...,375000.0,100957.0,Excepted Service Permanent,"Veterans Affairs, Veterans Health Administration",Per Year,2017-12-29,RN-16-DM-1659295-BU,"Los Angeles, California",2017-03-23,Physician (Thoracic Surgeon - Scientist),All individuals applying under this announceme...,Physician (Thoracic Surgeon - Scientist) | htt...,False,237978.5


In [147]:
AvgMinPay = salaried_location.groupby(["DepartmentName"])["MinPay"].mean().to_frame().sort_values("MinPay", ascending = False)
AvgMinPay["MinPay"] = AvgMinPay["MinPay"].round()
AvgMinPay.head(10)

,MinPay
DepartmentName,
National Aeronautics and Space Administration,97933.0
Department of the Treasury,96561.0
Department of Energy,95234.0
Department of Transportation,85475.0
Other Agencies and Independent Organizations,80937.0
Judicial Branch,79207.0
Department of Veterans Affairs,78947.0
Department of Agriculture,77678.0
Department of Health And Human Services,77194.0


In [148]:
AvgMedPay = salaried_location.groupby(["DepartmentName"])["MedPay"].mean().to_frame().sort_values("MedPay", ascending = False)
AvgMedPay["MedPay"] = AvgMedPay["MedPay"].round()
AvgMedPay.head(10)

,MedPay
DepartmentName,
Department of the Treasury,121064.0
Department of Energy,117858.0
National Aeronautics and Space Administration,115642.0
Department of Veterans Affairs,112786.0
Department of Transportation,109721.0
Department of Health And Human Services,109016.0
Judicial Branch,98995.0
Other Agencies and Independent Organizations,97454.0
Department of the Navy,95600.0


In [149]:
AvgMaxPay = salaried_location.groupby(["DepartmentName"])["MaxPay"].mean().to_frame().sort_values("MaxPay", ascending = False)
AvgMaxPay["MaxPay"] = AvgMaxPay["MaxPay"].round()
AvgMaxPay.head(10)

,MaxPay
DepartmentName,
Department of Veterans Affairs,146626.0
Department of the Treasury,145566.0
Department of Health And Human Services,140838.0
Department of Energy,140482.0
Department of Transportation,133967.0
National Aeronautics and Space Administration,133352.0
Department of the Navy,119795.0
Judicial Branch,118784.0
Other Agencies and Independent Organizations,113971.0


It looks like the **Dept of Energy** and the **Dept of Treasury** are consistently in the top 5 averages for pay. Let's just examine those departments to see which kinds of roles pay the most. 

### Department of Energy

In [150]:
energy = salaried_location[(salaried_location["DepartmentName"] == "Department of Energy") \
                               & (salaried_location["MedPay"] > 99999)]

energy = energy.sort_values("MedPay", ascending = False)
energy = energy.head(100)
energy = energy[["PositionTitle", "MedPay", "AllDescription", "JobGrade"]].reset_index(drop = True)
energy

,PositionTitle,MedPay,AllDescription,JobGrade
0,Assistant General Counsel for Electricity and ...,175259.5,Assistant General Counsel for Electricity and ...,ES
1,"Program Manager, Surface Ship Nuclear Propulsion",175259.5,"Program Manager, Surface Ship Nuclear Propulsi...",ES
2,"Program Manager, Surface Ship Nuclear Propulsion",175259.5,"Program Manager, Surface Ship Nuclear Propulsi...",ES
3,Attorney-Adviser (Public Utilities),146833.5,Attorney-Adviser (Public Utilities) | https://...,GS
4,Energy Industry Analyst,146833.5,Energy Industry Analyst | https://www.usajobs....,GS
5,Operations Manager,145563.5,Operations Manager | https://www.usajobs.gov:4...,GS
6,Attorney-Adviser (Public Utilities),128825.0,Attorney-Adviser (Public Utilities) | https://...,GS
7,Physical Security Specialist,128825.0,Physical Security Specialist | https://www.usa...,GS
8,Information Security Specialist (Senior Techni...,120212.5,Information Security Specialist (Senior Techni...,NQ
9,Biologist / General Engineer / Physical Scientist,117503.5,Biologist / General Engineer / Physical Scient...,EN


** Let's tokenize the job descriptions and remove the stop words ** 

In [151]:
energyDescriptions = np.array(energy["AllDescription"])

In [152]:
# remove all the stop words and make all the words lowercase
cv = CountVectorizer(stop_words = "english", lowercase = True)
tk = cv.build_analyzer()

allDescriptions = []

for d in energyDescriptions:
    tokens = tk(d)
    for t in tokens:
        allDescriptions.append(t)    
        
print("Total Words: ", len(allDescriptions))

Total Words:  5656


**Let's stem the words using PorterStemmer and then output the most frequent root words**

In [153]:
stemmer = PorterStemmer()
tokens_stem = []

for w in allDescriptions:
    s = stemmer.stem(w)
    tokens_stem.append(s)

wordFreq = nltk.FreqDist(tokens_stem)

print('{0:12s}: {1}'.format('Token', 'Count'))
print(25 * '-')

for token, freq in wordFreq.most_common(20):
    print('{0:12s}:  {1:4.3f}'.format(token, freq))

Token       : Count
-------------------------
experi      :  149.000
applic      :  64.000
abil        :  63.000
work        :  62.000
requir      :  58.000
program     :  58.000
posit       :  56.000
energi      :  55.000
nuclear     :  54.000
knowledg    :  53.000
legal       :  51.000
includ      :  51.000
secur       :  49.000
attorney    :  47.000
profession  :  47.000
law         :  44.000
manag       :  43.000
provid      :  41.000
resum       :  39.000
servic      :  38.000


Based on the scraped job postings for the Dept of Energy, there is a lot of emphasis placed on experience and management. Knowledge of nuclear energy and the legalities behind energy are also important. 

### Department of the Treasury

In [154]:
treasury = salaried_location[(salaried_location["DepartmentName"] == "Department of the Treasury") \
                               & (salaried_location["MedPay"] > 99999)]

treasury = treasury.sort_values("MedPay", ascending = False)
treasury = treasury.head(100)
treasury = treasury[["PositionTitle", "MedPay", "AllDescription", "JobGrade"]].reset_index(drop = True)
treasury

,PositionTitle,MedPay,AllDescription,JobGrade
0,"Financial Economist, NB-0110-6",154441.5,"Financial Economist, NB-0110-6 | https://www.u...",NB
1,"Financial Economist, NB-0110-VI",154441.5,"Financial Economist, NB-0110-VI | https://www....",NB
2,"Financial Economist, NB-0110-6",154441.5,"Financial Economist, NB-0110-6 | https://www.u...",NB
3,Associate Director (Quality),154137.5,Associate Director (Quality) | https://www.usa...,ES
4,"General Attorney-Advisor (Tax), Branch Chief (...",146833.5,"General Attorney-Advisor (Tax), Branch Chief (...",GS
5,"Director, Operations and Personnel",146833.5,"Director, Operations and Personnel | https://w...",GS
6,"Supervisory General Attorney (Tax), Branch Chi...",146833.5,"Supervisory General Attorney (Tax), Branch Chi...",GS
7,"Supervisory Trial Attorney (Tax), Associate Ar...",144257.5,"Supervisory Trial Attorney (Tax), Associate Ar...",GS
8,"Financial Economist, NB-0110-5.2",131665.0,"Financial Economist, NB-0110-5.2 | https://www...",NB
9,"Financial Economist, NB-0110-5.2",131665.0,"Financial Economist, NB-0110-5.2 | https://www...",NB


**Let's tokenize the job descriptions and remove the stop words**

In [155]:
treasuryDescriptions = np.array(treasury["AllDescription"])

In [156]:
# remove all the stop words and make all the words lowercase
cv = CountVectorizer(stop_words = "english", lowercase = True)
tk = cv.build_analyzer()

allDescriptions = []

for d in treasuryDescriptions:
    tokens = tk(d)
    for t in tokens:
        allDescriptions.append(t)    
        
print("Total Words: ", len(allDescriptions))

Total Words:  5981


**Let's stem the words using PorterStemmer and then output the most frequent root words**

In [157]:
stemmer = PorterStemmer()
tokens_stem = []

for w in allDescriptions:
    s = stemmer.stem(w)
    tokens_stem.append(s)

wordFreq = nltk.FreqDist(tokens_stem)

print('{0:12s}: {1}'.format('Token', 'Count'))
print(25 * '-')

for token, freq in wordFreq.most_common(20):
    print('{0:12s}:  {1:4.3f}'.format(token, freq))

Token       : Count
-------------------------
experi      :  163.000
econom      :  98.000
work        :  88.000
level       :  79.000
requir      :  78.000
posit       :  72.000
includ      :  56.000
special     :  55.000
gs          :  55.000
treasuri    :  53.000
servic      :  53.000
engin       :  53.000
year        :  52.000
depart      :  47.000
meet        :  44.000
financi     :  43.000
offic       :  40.000
follow      :  40.000
respons     :  39.000
date        :  38.000


Based on the scraped job listings, the Dept of the Treasury place a lot of emphasis on job experience and the knowledge of the economy. Not much else can be inferred from this list of word counts, however. Perhaps, we can get more information using n-grams. 